In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

In [ ]:
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# uncomment the following lines if youre running in a local machine with GPU
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
# Common imports
import numpy as np
import os

from collections import Counter
import tensorflow_datasets as tfds

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# O que é NLP?

O processamento de linguagem natural (Natural Language Processing - NLP) é um ramo da inteligência artificial que busca dar às máquinas o entendimento sobre a linguagem humana. À primeira vista, pode parecer uma tarefa simples, mas vários desafios são expostos quando entramos à fundo nesta tarefa. A linguagem humana é confusa para as máquinas, tendo em vista que sua construção é complexa. Existem estruturas hierárquicas que devem ser seguidas quando montamos uma frase, podem haver significados que fogem do que era esperado ao pé da letra, como o sarcasmo. Existem diversas fontes de ambiguidade na linguagem humana e é uma tarefa realmente complicada fazer as máquinas entenderem e gerarem alguma interpretação de valor a partir dela. E é neste ponto que o NLP, valendo-se das poderosas ferramentas desenvolvidas ao longo do tempo na área de IA, atua. Vale ressaltar que nem só de IA NLP vive, pois muitos dos modelos de NLP nasceram de abordagens que não envolviam redes neurais.

Os exemplos mais claros de onde NLP é aplicado podem ser vistos no dia-a-dia de cada um. Ferramentas como o Google Tradutor, Alexa, SIRI, assistente do Google, corretor automático dos teclados e a sugestão para completar automaticamente as palavras que escrevemos são algumas das aplicações mais famosas.

Como forma de trazer uma introdução a algumas das técnicas mais famosas que foram desenvolvidas nos últimos anos, separamos, neste post, algumas que são aplicadas a redes neurais voltadas a NLP e damos uma breve explicação sobre cada uma. Obviamente é complicado abordar todas em um único post, por isso daremos mais ênfase naquelas que mais se destacaram e que julgamos mais interessantes de serem abordadas.

## Neural Language Models

Uma das primeiras tarefas designadas a NLP foram os famosos “Language Models”, onde, com base nas palavras passadas, deve-se prever qual será a próxima palavra de uma sentença. Esta abordagem encontra sua aplicação, por exemplo, na ferramenta de completar automaticamente a próxima palavra ou corrigi-la em caso de erro de digitação. 

Como forma de cumprir essa tarefa, uma das primeiras redes neurais propostas para NLP, desenvolvida por Bengio e colaboradores em 2001, segue o modelo feedforward e tinha como entrada representações vetoriais das palavras. Tais representação são o que chamamos hoje de word embeddings, que falaremos sobre mais à frente. O modelo, **para esta tarefa**, apesar de simples, tem desempenho satisfatório mesmo quando comparado com modelos atuais um pouco mais complexos. Porém, para tarefas mais complexas, esse tipo de abordagem se torna ineficiente pelos motivos já descritos neste texto, pois conforme precisamos extrair interpretação dos dados que dispomos, apenas usar o texto bruto como entrada não é suficiente, o que implica na necessidade do desenvolvimento e utilização de modelos mais complexos e completos.

Para exemplificar, vamos recriar o modelo proposto por Bengio e colaboradores em 2001 que tem o objetivo de prever qual a próxima palavra de uma sentença dado um contexto anterior. Utilizaremos o framework Keras em conjunto com o Tensorflow para a criação do modelo. O conjunto de dados contendo as revisões do IMDB serão utilizados neste post. Esses dados são famosos no NLP, principalmente em tarefas com a finalidade de classificar a informação contida em um texto. Por exemplo, a sentença _"Este é um exemplo de filme genérico e chato"_  representa uma opinião negativa sobre um filme. Por outro lado, _"Esse filme é incrível!"_ é uma sentença associada à um sentimento positivo em relação ao filme. O conjunto de dados IMDB revisões é composto exatamente por sentenças positivas e negativas relacionadas a filmes, sendo comumente usado em tarefas de classificação, também denominadas dentro do NLP de tarefas de Análise de Sentimentos.

Como a tarefa NLP proposta por Bengio e colaboradores envolve a predição da próxima palavra de uma sentença e não a classificação de uma sentença, precisamos adaptar os textos contidos nas revisões do IMDB. A ideia geral do trabalho de Bengio e colaboradores é prever a próxima palavra após observar uma quantidade $n$ de palavras. Cada uma das $n$ palavras anteriores seriam representadas por um vetor numérico. Portanto, como entrada para o treinamento do modelo, precisamos de uma palavra e de suas $n$ palavras anteriores. Para adaptar os dados do IMDB à tarefa de Language Modeling vamos dividir cada revisão do IMDB em janelas de 11 palavras. Em cada janela, a última palavra é àquela a ser predita e as 10 anteriores representam o contexto.

A seguir importamos o conjunto de dados de revisões do IMDB e mostramos duas sentenças contidas nele com as respectivas classes (Positivo ou Negativo). Note que os textos encontram-se muito poluídos, ainda precisamos excluir pontuações desnecessárias, caracteres da linguagem html, como a tag `<br/>`, e outros artefatos que podem atrapalhar nosso modelo. Isso será feito por meio da função `preprocess()`.

In [ ]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUEEAX4/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUEEAX4/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteUEEAX4/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positivo" if label else "= Negativo")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negativo

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negativo



In [ ]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.lower(X_batch)
    X_batch = tf.strings.split(X_batch)
    
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

Como o intuito é ser sucinto e demonstrar como os modelos de NLP evoluíram ao incluir características adicionais, vamos somente utilizar uma amostragem de 5% das 25.000 sentenças de revisão contidas no subconjunto de treinamento do dataset do IMDB (`datasets["train"]`). O próximo código faz essa amostragem e divide cada revisão em janelas de 11 palavras mencionadas que serão usadas como entrada para o modelo. Ao final, criamos o objeto do tipo `Dataset` que alimentará o modelo em batchs de tamanho 32 oferecendo as 10 palavras, seguido pela última palavra da janela, isto é, àquela que deseja-se prever.

In [ ]:
n_steps = 10
window_length = n_steps + 1
batch_size = 32

datasets_list = []
vocabulary = Counter()

for X_batch, y_batch in datasets["train"].batch(1).take(int(train_size*0.05)):
    X_b, y_b = preprocess(X_batch, y_batch)
    dsu = tf.data.Dataset.from_tensor_slices(X_b[0])
    dsu = dsu.window(window_length, shift=1, drop_remainder=True)
    dsu = dsu.flat_map(lambda window: window.batch(window_length))
    for x in dsu.as_numpy_iterator():
        datasets_list.append(x)
        vocabulary.update([s.lower() for s in list(x)])
        

O universo de palavras dos nossos dados, isto é, o tamanho do vocabulário é de 41.422 palavras. Esse fato é um entrave durante o cálculo da função de perda e um modo de superá-lo é reduzir o tamanho do vocabulário. Manteremos somente as 10.000 palavras mais frequentes nas revisões e, em seguida, codificaremos cada uma por um índice.

In [ ]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

Por exemplo, os 4 índices representando as palavras da frase _"This movie was faaaaaantastic"_ são:

In [ ]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"This movie was faaaaaantastic".split():
    print(word_to_id.get(word) or vocab_size)

10000
17
11
10000


Além disso, criamos um dicionário para armazenar o mapeamento entre o índice e a palavra e geramos a tabela lookup que tem o papel de transformar as palavras de uma sentença nos índices correspondentes como podemos ver no exemplo abaixo.

In [ ]:
id_to_word = {}
for k,v in word_to_id.items():
    id_to_word[v] = k

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[10745,    17,    11, 10053]])>

In [ ]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), table.lookup(y_batch)

In [ ]:
batch_size = 32
train_set = tf.data.Dataset.from_tensor_slices(datasets_list).shuffle(10000).batch(batch_size)
train_set = train_set.map(lambda windows: (windows[:, :-1], windows[:, -1]))
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
for x in train_set.take(1):
    print("Um batch com sentenças codificadas em índices: ", x[0])

Um batch com sentenças codificadas em índices:  tf.Tensor(
[[  451   425    29    87   526   407  2629    20   194   674]
 [   10   386     2  1575   179     2   169     4     0   580]
 [   72  2388   258     1     0   597   959     1     0   387]
 [   37   483  5813   205    20    72   148    13    10   230]
 [ 5847    34   173    73  3166     0   112    14   437   334]
 [   10     5   506   612     4     2  2191     3   182   563]
 [    3     0  2494 10885     3   161   313     8  1210    10]
 [  128  2198     0   886    96     3     0  7728     1  7729]
 [10967 10290     0  2548     3     0   203     4    23  1967]
 [   19   142    41     9   952     0   341     5 10206  5923]
 [   42     4    23 10760   237  1258     6   114   893   598]
 [   19    26     3    55  4616    13   216    64   233   372]
 [    0   450     9   134  3437     6    10    16    20   571]
 [10888    27    52    39    19    91   142   114   488   164]
 [    5   260    30    26     3     0    86   522   174  11

A importância do trabalho de Bengio e colaboradores em 2001 ressaltou-se, além da simplicidade, pelas técnicas aplicadas que se tornariam essenciais em inúmeras tarefas de NLP. Dentre as técnicas está o uso de word embeddings. Bengio e colaboradores desenvolveram uma arquitetura de rede neural que inicia-se a partir da concatenção dos vetores representativos das $n$ palavras anteriores. Estes são, então, passados para a camada intermediária que aplica uma função não linear ao objeto concatenado e, em seguida, a rede produz a probabilidade de prever cada palavra por meio de uma função softmax.

image.png

_Figure 1: Classic neural language model (Bengio et al., 2003)_

In [ ]:
# como entrada usamos as 10 palavras anteriores
regular_inputs = keras.layers.Input(shape=[10])

# we check the index of these words
# cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(regular_inputs)

# get the embedding vector for each word
cat_embedding = keras.layers.Embedding(input_dim=vocab_size+num_oov_buckets, output_dim=30)(regular_inputs)

# produce the concatenated previous word embedding vec
encoded_inputs = tf.keras.layers.Flatten()(cat_embedding)

# camada intermediária
intermediate_layer = keras.layers.Dense(128, activation="tanh")(encoded_inputs)

# vetor com prob para cada palavra
outputs = keras.layers.Dense(vocab_size + num_oov_buckets, activation="softmax")(intermediate_layer)

In [ ]:
model = keras.models.Model(inputs = [regular_inputs], outputs=[outputs])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(train_set, epochs=20)

Epoch 1/20
8529/8529 [==============================] - 49s 5ms/step - loss: 6.6464
Epoch 2/20
8529/8529 [==============================] - 50s 6ms/step - loss: 6.0035
Epoch 3/20
8529/8529 [==============================] - 51s 6ms/step - loss: 5.6519
Epoch 4/20
8529/8529 [==============================] - 50s 6ms/step - loss: 5.3530
Epoch 5/20
8529/8529 [==============================] - 51s 6ms/step - loss: 5.0843
Epoch 6/20
8529/8529 [==============================] - 51s 6ms/step - loss: 4.8418
Epoch 7/20
2306/8529 [=======>......................] - ETA: 36s - loss: 4.6427

A nossa reimplementação do modelo de Bengio e colaboradores tem o caráter didático. O intuito é demonstrar como podemos codificar as técnicas de NLP e como estas são incluídas nos modelos. Portanto, esse post não tem o intuito de treinar modelos com alta capacidade preditiva pois sofre com algumas limitações, tais como conjunto de dados pequeno e quantidade de épocas de treinamento insuficientes. Apesar das limitações, vamos mostrar abaixo, para fins de exemplificação, como fazer predições, isto é, criar uma revisão de filme usando o nosso modelo treinado.

In [ ]:
def next_word(text):
    X_new = tf.strings.regex_replace(text, rb"<br\s*/?>", b" ")
    X_new = tf.strings.regex_replace(X_new, b"[^a-zA-Z']", b" ")
    X_new = tf.strings.split(X_new)
    enc_X = table.lookup(X_new)
    enc_X = enc_X[tf.newaxis, :]
    y_proba = model.predict(enc_X)
    rescaled_logits = tf.math.log(y_proba)
    word_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    w = (word_id[0][0]).numpy()
    if w > 10000:
        w = 0
    return " "+ tf.compat.as_str_any(id_to_word[w])

In [ ]:
def complete_text(text, n_words = 50):
    for wi in range(n_words):
        text_in = text.split(" ")[wi:wi+10]
        text_in = " ".join(text_in)
        assert (len(text_in.split(" ")) == 10)
        text += next_word(text_in)
    return text

In [ ]:
complete_text("The movie was fantastic I'd like to watch it again")

"The movie was fantastic I'd like to watch it again painting is this a them cameron not of and that and villain show and i friend her heart and the professor a audience black at and life lot is of great live it been red football movie i girls we everyone images but or which see and the go as"

## Multitask Learning
	
Para este tópico, queremos que o leitor pense na seguinte pergunta: se a linguagem é uma só, será que redes neurais para NLP precisam, em todo treinamento, aprender as relações linguísticas de baixo nível do zero? 
	
Bom, teoricamente sim, porém, com o intuito de acelerar o treinamento que seria muito demorado, técnicas de multitask learning foram desenvolvidas. Tais técnicas consistem em compartilhar parâmetros entre diferentes modelos, encorajando-os a aprender representações que são úteis para diversas tarefas. Através disso, é possível fazer com que os modelos compartilhem a informação linguística de baixo nível, que, normalmente, é o que representa a maior parte dos parâmetros do modelo.
	
Surgindo de uma ideia, essa abordagem fez ser possível, por exemplo, o uso de word embeddings pré-treinadas, o que aumenta consideravelmente a performance dos modelos e reduz significativamente o tempo gasto no treinamento. Atualmente, tal técnica é usada em várias aplicações de NLP.

## Word Embedding

Mas como representar as palavras de uma forma que os computadores entendam? É nesta questão que as famosas Word Embeddings encontram seu trabalho. A principal ideia por trás dessa técnica é: palavras similares geralmente são usadas em contextos similares. Dessa forma, técnicas como o word2vec são aplicadas de forma a extrair a relação entre palavras tais como gênero, tempo verbal e até relações de país-capital. Tais relações são construídas, basicamente, de representações vetoriais das palavras em um espaço n-dimensional, onde a posição de um vetor com relação ao outro representa algum tipo de relação entre as palavras e é através dessas relações que as máquinas conseguem atribuir algum tipo de significado às palavras.

A consagração desta técnica no campo de NLP se deu na melhora de performance mostrada pelos modelos quando inicializados com word embeddings pré-treinadas para diferentes tipos de tarefas.

A seguir vamos mostrar como é simples incrementar o nosso modelo recriado de Bengio e colaboradores para utilizar word embeddings pré-treinadas. Como o embedding a ser utilizado foi desenvolvido a partir de outro modelo que, possivelmente, aplicava outras etapas de pré-processamento dos textos, vamos criar nosso objeto `Dataset` de forma mais simples.

In [ ]:
def preprocess(X_batch, y_batch):
    #X_batch = tf.strings.substr(X_batch, 0, 300)
    #X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    #X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

n_steps = 10
window_length = n_steps + 1

datasets_list = []
outputs_list = []
vocabulary = Counter()

#for X_batch, y_batch in datasets["train"].batch(1).take(2):
for X_batch, y_batch in datasets["train"].batch(1).take(int(train_size*0.05)):
    # print(X_batch.numpy(), y_batch.numpy())
    X_b, y_b = preprocess(X_batch, y_batch)
    # print(X_b.numpy(), y_b.numpy())
    dsu = tf.data.Dataset.from_tensor_slices(X_b[0])
    dsu = dsu.window(window_length, shift=1, drop_remainder=True)
    dsu = dsu.flat_map(lambda window: window.batch(window_length))
    # print("DEPOIS")
    for x in dsu.as_numpy_iterator():
      #print(x, end=' XX')
        datasets_list.append(' '.join([s.decode("utf-8") for s in x[:-1]]))
        outputs_list.append(x[-1])
        vocabulary.update(list(x))


In [ ]:
import tensorflow_hub as hub

embed_nnlm = hub.load("https://tfhub.dev/google/nnlm-en-dim50/2")
embeddings = embed_nnlm(["cat is on the mat", "dog is in the fog"])

Como usamos um embedding pré-treinado, aplicamos a transformação das palavras em vetores durante a criação do objeto `Dataset` que será a entrada para o modelo. Dessa maneira, a rede fica ainda mais simples, possuindo, apenas, a camada densa e a última camada com softmax.

In [ ]:
def encode_words_preemb(X_batch, y_batch):
    return embed_nnlm(X_batch), table.lookup(y_batch)

batch_size = 32
train_set_X = tf.data.Dataset.from_tensor_slices(datasets_list)
train_set_y = tf.data.Dataset.from_tensor_slices(outputs_list)
train_set = tf.data.Dataset.zip((train_set_X, train_set_y))
train_set = train_set.batch(batch_size).map(encode_words_preemb).prefetch(1)

In [ ]:
string_inputs = keras.layers.Input(shape=[50])

intermediate_layer = keras.layers.Dense(128, activation="tanh")(string_inputs)

outputs = keras.layers.Dense(vocab_size + num_oov_buckets, activation="softmax")(intermediate_layer)

In [ ]:
model = keras.models.Model(inputs = [string_inputs], outputs=[outputs])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(train_set, epochs=20)

Epoch 1/20
8833/8833 [==============================] - 9s 1ms/step - loss: 6.9563
Epoch 2/20
8833/8833 [==============================] - 9s 1ms/step - loss: 6.6709
Epoch 3/20
8833/8833 [==============================] - 9s 1ms/step - loss: 6.5648
Epoch 4/20
8833/8833 [==============================] - 9s 999us/step - loss: 6.4931
Epoch 5/20
8833/8833 [==============================] - 9s 981us/step - loss: 6.4418
Epoch 6/20
8833/8833 [==============================] - 9s 981us/step - loss: 6.4042
Epoch 7/20
8833/8833 [==============================] - 9s 978us/step - loss: 6.3756
Epoch 8/20
8833/8833 [==============================] - 9s 979us/step - loss: 6.3532
Epoch 9/20
8833/8833 [==============================] - 9s 983us/step - loss: 6.3349
Epoch 10/20
8833/8833 [==============================] - 9s 989us/step - loss: 6.3192
Epoch 11/20
8833/8833 [==============================] - 9s 992us/step - loss: 6.3053
Epoch 12/20
8833/8833 [==============================] - 9s 986us/ste

In [ ]:
def next_word(text):
    enc_X = embed_nnlm([text])
    #print(enc_X)
    y_proba = model.predict(enc_X)
    rescaled_logits = tf.math.log(y_proba)
    word_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    w = (word_id[0][0]).numpy()
    if w > 10000:
        w = 0
    return " "+ tf.compat.as_str_any(id_to_word[w])

def complete_text(text, n_words = 50):
    for wi in range(n_words):
        text_in = text.split(" ")[wi:wi+10]
        text_in = " ".join(text_in)
        #print(text_in)
        assert (len(text_in.split(" ")) == 10)
        text += next_word(text_in)
    return text

In [ ]:
complete_text("The movie was fantastic I'd like to watch it again")

"The movie was fantastic I'd like to watch it again needs the is also finnerty a knows than continues killed bad the and cast as the the doing feel that doing perfect the did who to scattered the did does might vile provides ever handful man draw love first movie have a better acting hooten but a but face again"

## Neural Networks for NLP

Como forma de se ter a melhor performance em NLP, foram desenvolvidas arquiteturas de redes neurais para esta tarefa. Dentre os principais tipos, destacamos as redes neurais recorrentes (RNNs), as redes neurais convolucionais (CNNs) e as redes neurais recursivas.

Em redes feedforward, a informação sempre vai em uma única direção, da entrada para a saída. Em RNNs, a saída de uma camada é passada como entrada para a próxima camada e, também, pode ser utilizada como entrada para camadas passadas ou para ela mesma. Desta forma, a rede consegue manter o contexto dos dados ao invés de simplesmente passá-los pelas camadas, o que é uma abordagem muito benéfica para nossa missão, dado que a construção de uma sentença depende de todo o contexto e não apenas das palavras próximas àquela que estamos olhando. Um lado negativo dessas redes é o seu treinamento lento e a sensibilidade a problemas como o vanishing e exploding gradient. Redes como as Long short-term memory (LSTMs) são utilizadas de forma a minimizar estes problemas, mas com o custo de um treinamento um pouco mais lento.

As CNNs, quando utilizadas em NLP, operam em duas dimensões apenas e os filtros só precisam se mover na dimensão temporal. Uma vantagem quanto ao tempo de treinamento de CNNs se dá pelo fato de serem mais paralelizáveis que as RNNs, uma vez que seu treinamento depende apende do contexto local e não de estados passados. Dessa forma, é possível fazer a combinação das duas e obter resultados ainda mais satisfatórios do que quando implementadas sozinhas.

É notável que as RNNs e CNNs assumem a linguagem como uma sequência e utilizam isso em seu favor. Porém, como dito na introdução deste texto, a linguagem apresenta uma estrutura hierárquica, dado que as palavras são usadas em frases de maior ordem, que, por sua vez, são combinadas de acordo com as regras gramaticais da língua em questão. Nesse quesito, as redes neurais recursivas tratam a linguagem não como uma sequência, mas como uma estrutura em árvore, onde as palavras têm suas representações feitas de baixo para cima, em vez da esquerda para a direita.

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, 10],
                     dropout=0.2, recurrent_dropout=0.2),

    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    
    keras.layers.TimeDistributed(keras.layers.Dense(vocab_size,
                                                    activation="softmax"))
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(train_set, epochs=20)

Epoch 1/20
8814/8814 [==============================] - 9s 992us/step - loss: 6.4890
Epoch 2/20
8814/8814 [==============================] - 9s 978us/step - loss: 6.4118
Epoch 3/20
8814/8814 [==============================] - 9s 985us/step - loss: 6.3653
Epoch 4/20
8814/8814 [==============================] - 9s 997us/step - loss: 6.3303
Epoch 5/20
8814/8814 [==============================] - 9s 1ms/step - loss: 6.3018
Epoch 6/20
8814/8814 [==============================] - 9s 1ms/step - loss: 6.2774
Epoch 7/20
8814/8814 [==============================] - 9s 1ms/step - loss: 6.2557
Epoch 8/20
8814/8814 [==============================] - 9s 1ms/step - loss: 6.2357
Epoch 9/20
8814/8814 [==============================] - 9s 994us/step - loss: 6.2170
Epoch 10/20
8814/8814 [==============================] - 9s 1ms/step - loss: 6.1990
Epoch 11/20
8814/8814 [==============================] - 9s 1ms/step - loss: 6.1817
Epoch 12/20
8814/8814 [==============================] - 9s 1ms/step - loss

In [ ]:
complete_text("The movie was fantastic I'd like to watch it again")

"The movie was fantastic I'd like to watch it again to and he had i couldn't guys one think plot of true one say as if funny only and cast the about almost the confidence the to a and woman i all trying the see special before just entertaining the and is the me reference the jerusalem will in together"

## Sequence-to-sequence models

Outra abordagem desenvolvida com o passar do tempo e utilizada em NLP são os sequence-to-sequence models, um framework feito para mapear uma sequência de dados para outra sequência utilizando redes neurais. Nesta abordagem, temos uma rede representando um encoder, ou seja, uma parcela da rede que comprime a sequência de entrada em uma representação vetorial com menos dimensões. Em seguida, a rede que representa o decoder tem como entrada essa representação comprimida e, a partir dela, é capaz de prever outra sequência de dados na saída. Uma grande aplicação deste tipo de técnica se encontra na tradução entre línguas, em que uma frase na língua X é a entrada e a saída é a mesma frase na língua Y. 

O decoder pode não apenas ser condicionado a uma sequência, mas, também, a representações arbitrárias, o que permite gerar, por exemplo, uma legenda com base em uma imagem, e as aplicações não se limitam somente a isso. Para a implementação do encoder e do decoder, geralmente as RNNs são utilizadas, mas nada impede que outros tipos de arquiteturas sejam utilizados.

## Attention 

Como forma de trazer uma intuição sobre Attention, começaremos falando sobre a principal limitação de aprendizado utilizando sequence-to-sequence, que é o fato de que ele precisa comprimir todo o conteúdo da fonte em um vetor de tamanho fixo. Attention flexibiliza isto, permitindo que o decoder possa olhar para os estados passados da fonte, que, após o devido tratamento, são fornecidos como entradas para o decoder.

De forma mais simples, Attention permite que o modelo utilize apenas as partes da entrada que são relevantes para uma saída particular, daí o seu nome, uma vez que a atenção do modelo é direcionada para partes específicas da entrada. Com esse direcionamento, o Attetion proporciona uma maior eficiência aos modelos de NLP. 

## Pre trained language models  

Words Embeddings pré-treinadas podem ser usadas em praticamente todas as tarefas e são usadas apenas nas primeiras camadas dos modelos. Vale ressaltar que “Language models” precisam apenas de textos para serem treinados, pois o treinamento é feito de forma não supervisionada, o que significa que o treinamento pode conter milhões de tokens, inclusive em línguas diferentes. “Language models” pré-treinados tem seu destaque por possibilitar que o treinamento seja feito utilizando muito menos dados, o que se resume em economia de tempo e de dinheiro, sendo assim, é uma técnica bastante utilizada no campo de NLP.

## Conclusão

Neste post, buscamos trazer uma introdução teórica e prática sobre o que é o NLP e como algumas de suas principais ferramentas, que se desenvolveram ao longo dos últimos anos, funcionam. Hoje em dia, NLP encontra aplicações nas mais diversas áreas econômicas e de pesquisas e é uma das aplicações de IA mais presentes no nosso dia a dia como exposto na introdução, sendo assim, fica fácil notar sua relevância no cenário de pesquisa atual. Ressaltamos que este post é apenas introdutório, portanto as informações passadas requerem um estudo mais aprofundado para que seja possível o pleno entendimento, bem como recomendamos que o leitor trate com zelo especial a implementação prática para que a intuição sobre o que acontece quando se trabalha com NLP possa ser construída, que, afinal de contas, é o que importa.

Dessa forma, acreditamos que o primeiro contato com alguns dos principais termos e códigos que utilizamos quando se trabalha com NLP puderam ser apresentados ao leitor. A área apresenta muitos tópicos a serem estudados, então esperamos que o leitor possa se sentir incentivado a buscar mais informações e conhecimento sobre a área. 

## Referências

[1] Bengio, Y., Ducharme, R., & Vincent, P. (2001). Proceedings of NIPS.

[2] “A Review of the Recent History of Natural Language Processing.” Sebastian Ruder, 1 Oct. 2018, https://ruder.io/a-review-of-the-recent-history-of-nlp/.

[3] Géron, Aurélien. Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems. Second edition, O’Reilly Media, Inc, 2019.